Ion-acoustic waves in Jupyter with MPI

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import animation, rc
plt.rc('image', origin='lower', interpolation='nearest')
from IPython.display import HTML

In [ ]:
from ipyparallel import Client
c = Client(profile="mpi")

In [ ]:
%%px
from skeletor import cppinit, Float, Float2, Grid, Field, Particles, Sources, Ohm
import numpy
from mpi4py import MPI
import matplotlib.pyplot as plt
from mpi4py.MPI import COMM_WORLD as comm

# Quiet start
quiet = True

# Number of grid points in x- and y-direction
nx, ny = 32, 32

# Average number of particles per cell
npc = 256
assert(int(numpy.sqrt(npc)) % 1 = 0)
# Particle charge and mass
charge = 1.0
mass = 1.0

# Thermal velocity of electrons in x- and y-direction
vtx, vty = 0.0, 0.0
# Velocity perturbation of ions in x- and y-direction
vdx, vdy = 0.001, 0.001

# Timestep
dt = 0.1

# Total number of particles in simulation
npar = npc*nx*ny

# Wavenumbers
kx = 2*numpy.pi/nx
ky = 2*numpy.pi/ny

if quiet:
    # Uniform distribution of particle positions (quiet start)
    dx = 1/int(numpy.sqrt(npc))
    dy = dx
    X = numpy.arange(0,nx,dx)
    Y = numpy.arange(0,ny,dy)
    x, y = numpy.meshgrid(X, Y)
    x = x.flatten()
    y = y.flatten()
else:
    x = nx*numpy.random.uniform(size=npar).astype(Float)
    y = ny*numpy.random.uniform(size=npar).astype(Float)

# Normal distribution of particle velocities
vx = vdx*numpy.sin(kx*x) + vtx*numpy.random.normal(size=npar).astype(Float)
vy = vdy*numpy.sin(kx*y) + vty*numpy.random.normal(size=npar).astype(Float)

In [ ]:
%%px
# Start parallel processing
idproc, nvp = cppinit(comm)

# Concatenate local arrays to obtain global arrays
# The result is available on all processors.
def concatenate(arr):
    return numpy.concatenate(comm.allgather(arr))

# Create numerical grid. This contains information about the extent of
# the subdomain assigned to each processor.
grid = Grid(nx, ny, comm)

# Maximum number of electrons in each partition
npmax = int(1.5*npar/nvp)

# Create particle array
ions = Particles(npmax, charge, mass)

# Assign particles to subdomains
ions.initialize(x, y, vx, vy, grid)


In [ ]:
%%px
# Set the force to zero (this will of course change in the future).
fxy = Field(grid, comm, dtype=Float2)
fxy.fill(0.0)

# Initialize sources
sources = Sources(grid, comm, dtype=Float)

# Initialize Ohm's law solver
ohm = Ohm(grid, temperature=1.0, charge=charge)

# Calculate initial density and force

# Deposit sources
sources.deposit(ions)
# Adjust density (we should do this somewhere else)
sources.rho /= npc
sources.rho.add_guards()
sources.rho.copy_guards()

# Calculate forces (Solve Ohm's law)
ohm(sources.rho, fxy, destroy_input=False)

In [ ]:
%px rho_global = np.concatenate(comm.allgather(sources.rho.trim()))
fig = plt.figure()
im = plt.imshow(c[0]['rho_global'], animated=True,vmin=0.999, vmax=1.001)

In [ ]:
%%capture
def updatefig(*args):
    # Push particles on each processor. This call also sends and
    # receives particles to and from other processors/subdomains. The
    # latter is the only non-trivial step in the entire code so far.
    %px ions.push(fxy, dt)

    # Deposit sources
    %px sources.deposit(ions)
    # Adjust density (we should do this somewhere else)
    %px sources.rho /= npc
    %px sources.rho.add_guards()
    %px sources.rho.copy_guards()
    
    # Calculate forces (Solve Ohm's law)
    %px ohm(sources.rho, fxy, destroy_input=False)
    %px rho_global = np.concatenate(comm.allgather(sources.rho.trim()))
    im.set_array(c[0]['rho_global'])
    im.autoscale()
    return im,
    
anim = animation.FuncAnimation(fig, updatefig, frames=200, blit=True)

In [ ]:
%%capture
HTML(anim.to_html5_video())

In [ ]:
%%capture
anim.save('animation.mp4', writer='ffmpeg', fps=60)